In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ipywidgets as ipyw
out = ipyw.Output(layout={'border': '100px solid black'})
out

Output(layout=Layout(border='100px solid black'))

In [3]:
import ipywidgets as ipyw

label_layout = ipyw.Layout(width='100px')

class WrappedToggles(ipyw.HBox):
    
    def __init__(self):
        layout = ipyw.Layout(width='400px')
        super().__init__(options=['1','2'], layout=layout)

    def _ipython_display_(self, **kwargs):
        label = 'Label:'
        box_label = ipyw.HTML(value=label, layout=label_layout)
        children = (box_label, self)
        ipyw.HBox(children=children)._ipython_display_()
# Can pass to children
# Can disable sync and link two widgets together
WrappedToggles()

In [2]:
class Shout(object):
    def __init__(self, text):
        self.text = text

    def _repr_html_(self):
        return "<h1>" + self.text + "</h1>"
Shout("test")

In [1]:
import re
import ipywidgets as ipw
from traitlets import Unicode
from IPython.display import display


class IntMultipliedDropdown(ipw.HBox):
    _VALUE_PATTERN = re.compile('(?P<num>\d+) (?P<option>\w+-?\w*)')
    value = Unicode()

    def __init__(self, options, option_value, int_value=1, **kwargs):
        self.number = ipw.IntText(int_value)
        self.options = ipw.Dropdown(options=options, value=option_value)

        self._update_value()

        self.number.observe(self._update_value, names='value')
        self.options.observe(self._update_value, names='value')
        self.observe(self._update_children, names='value')

        super().__init__(children=[self.number, self.options], **kwargs)


    def _update_children(self, *args):
        match = re.search(self._VALUE_PATTERN, self.value)
        groupdict = match.groupdict()
        self.number.value = groupdict['num']
        self.options.value = groupdict['option']

    def _update_value(self, *args):
        self.value = "{} {}".format(self.number.value, self.options.value)

mywidget = IntMultipliedDropdown(['apple', 'bed', 'cell'], 'cell')
display(mywidget)

IntMultipliedDropdown(children=(IntText(value=1), Dropdown(index=2, options=('apple', 'bed', 'cell'), value='c…

In [9]:
import pandas as pd
import sys
from io import StringIO
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
tab = widgets.Tab()    
out = widgets.Output(layout={'border': '1px solid black'})
up = widgets.FileUpload(accept="", multiple=False)
delim = widgets.RadioButtons(
    options=[';', ',', ' '],
    description='Separator: ',
    disabled=False)
eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)
rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to Upload',
    icon='check')
button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search')
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='danger',
    tooltip='Click to Plot',
    icon='pencil')
graph_type = widgets.Dropdown(
    options=['Bar Chart', 'Line Chart'],
    value='Bar Chart',
    description='Chart Type:',
    disabled=False)
x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)
toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])
accordion = widgets.Accordion(children=[
    up, 
    widgets.VBox([delim, eraser]), 
    rows])
accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')
accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload]),
    out
])
children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    widgets.VBox([
        widgets.HBox([graph_type, color_picker]),
        widgets.HBox([x_axis, y_axis]), 
        button_plot,
        out
])]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")
tab
def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
#             typ = up_value[i]["metadata"]["type"] if typ == "text/csv":
            content = up_value[i]["content"]
            content_str = str(content, 'utf-8')
    
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value)    
        return df
    else:
        return None
def preview():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n -----Now this is how your DF looks like:----- \n')
        if df is not None:
            print(df.head(10))
        else:
            print('Configuration is wrong/missing...')
def upload():
    df = df_converter()
    with out:
        out.clear_output()
        print('\n --------Your uploaded DF looks like:-------- \n')
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
        else:
            print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    print('Configuration is wrong/missing...')
    
    
def plot():
    graph = graph_type.value
    if graph != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                graph))
            if (df is not None):
                df = df.head(5)
                height = df[y_axis.value]
                bars = df[x_axis.value]
                y_pos = np.arange(len(height))
                plt.figure(figsize=(10,4))
                if graph == 'Bar Chart':
                    plt.bar(
                        y_pos, 
                        height, 
                        color=color_picker.value)
                    plt.xticks(y_pos, bars)
                elif graph == 'Line Chart':
                    plt.plot(
                        bars,
                        height, 
                        color=color_picker.value,
                        marker='o', 
                        linestyle='solid'
                    )
                    plt.xticks(bars)
                plt.show()
                
def preview_clicked(b):
    preview()
def upload_clicked(b):
    upload()
def desc_clicked(b):
    desc()
def plotter_clicked(b):
    plot()
    
button_preview.on_click(preview_clicked)
button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')
button_plot.on_click(plotter_clicked)
tab

In [5]:
if 1:
    acc = widgets.Accordion(children=[widgets.VBox(layout=widgets.Layout(max_width='610px',
                                                                         min_width='610px'))],
                            layout=widgets.Layout(width='auto',
                                                  max_width='650px',
                                                  max_height='500px',
                                                  overflow_y='scroll',
                                                  overflow_x='hidden'))
    acc.set_title(0, 'IBMQ Jobs')
    acc.selected_index = None
    acc.layout.visibility = 'hidden'
    display(acc)
    acc._dom_classes = ['job_widget']
    display(Javascript("""$('div.job_widget')
        .detach()
        .appendTo($('#header'))
        .css({
            'z-index': 999,
             'position': 'fixed',
            'box-shadow': '5px 5px 5px -3px black',
            'opacity': 0.95,
            'float': 'left,'
        })
        """))
    acc.layout.visibility = 'visible'


Accordion(children=(VBox(layout=Layout(max_width='610px', min_width='610px')),), layout=Layout(max_height='500…

NameError: name 'Javascript' is not defined

In [8]:
%%javascript
def modal_dialog(self,title,body):
    data = """
        require(
            ["base/js/dialog"],
            function(dialog) {
                dialog.modal({
                    title: '%s',
                    body: '%s',
                    sanitize: false,
                    buttons: {
                        'Close': {}
                    }
                });
            }
        )
     """ % (title,body)

display.display(Javascript(data=data,css='modal.css'))


<IPython.core.display.Javascript object>